In [1]:
%load_ext autoreload
%autoreload 2
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr2tapn/')
print(os.getcwd())

/home/vco/Projects/pm4py-dcr2tapn/notebooks_projects
/home/vco/Projects/pm4py-dcr2tapn


In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from copy import deepcopy
from pm4py.objects.dcr.obj import dcr_template
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.exporter import exporter as dcr_exporter
from pm4py.objects.dcr.utils.utils import nested_groups_and_sps_to_flat_dcr

In [3]:
from pm4py.objects.conversion.dcr.variants.to_timed_arc_petri_net import Dcr2TimedArcPetri

example = 'models/RunningExampleTAPNoriginal.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
dcr_dict = dcr_dict.obj_to_template()
dcr2tapn = Dcr2TimedArcPetri(debug=False,preoptimize=True, postoptimize=True)
tapn, m = dcr2tapn.apply(dcr_dict,tapn_path='models/original.tapn')
# master_df

In [4]:
from pm4py.objects.conversion.dcr.variants.to_timed_arc_petri_net import Dcr2TimedArcPetri

example = 'models/RunningExampleTAPNoriginal-livelock.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
dcr_dict = dcr_dict.obj_to_template()
dcr2tapn = Dcr2TimedArcPetri(debug=False,preoptimize=True, postoptimize=True)
tapn, m = dcr2tapn.apply(dcr_dict,tapn_path='models/original-livelock.tapn')
# master_df

In [5]:
from pm4py.objects.conversion.dcr.variants.to_timed_arc_petri_net import Dcr2TimedArcPetri

example = 'models/RunningExampleTAPNMilestone.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
dcr_dict = dcr_dict.obj_to_template()
dcr2tapn = Dcr2TimedArcPetri(debug=False,preoptimize=True, postoptimize=True)
tapn, m = dcr2tapn.apply(dcr_dict,tapn_path='models/milestone-deadlock.tapn')
# master_df

In [ ]:
# from pm4py.objects.conversion.dcr_apt.variants.to_inhibitor_net import Dcr2PetriNet
# example = 'models/RunningExampleTAPNMilestone.xml'
# dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
# # dcr_dict = nested_groups_and_sps_to_flat_dcr(dcr_dict)
# # dcr_exporter.apply(dcr_dict, 'models/debug_exported.xml')
# dcr_dict = dcr_dict.obj_to_template()
# d2p = Dcr2PetriNet(debug=True,preoptimize=True, postoptimize=True)
# net, marking, master_df = d2p.apply(dcr_dict,tapn_path='models/deadlock-pass.tapn')

In [3]:
from pm4py.objects.conversion.dcr_apt.variants.to_timed_arc_petri_net import Dcr2TimedArcPetri

example = 'models/debug.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
dcr_dict = dcr_dict.obj_to_template()
dcr2tapn = Dcr2TimedArcPetri(debug=False,preoptimize=True, postoptimize=False)
tapn, m, master_df = dcr2tapn.apply(dcr_dict,tapn_path='models/debug.tapn')
master_df

A  B               
    No In Re_inf Rex_inf
B 0  0  3      4       0
  1  0  3      2       0
A 0  0  2      4       1
  1  0  2      2       1
  2  0  2      0       1
  3  0  4      0       5
  4  0  4      0       1
  5  0  4      0       3

In [ ]:
dcr_dict

In [ ]:
from pm4py.objects.conversion.dcr_apt.variants.to_timed_arc_petri_net_submodules import timed_exceptional_cases, timed_preoptimizer
from pm4py.objects.conversion.dcr_apt.variants.to_timed_arc_petri_net import Dcr2TimedArcPetri

In [ ]:
G = deepcopy(dcr_template)
G['events'] = {'A','B'}
G['marking']['included'] = {'A', 'B'}
G['milestonesFor']['A'] = {'B'}
G['responseTo']['A'] = {'B'}
dcr2tapn = Dcr2TimedArcPetri(debug=False,preoptimize=True, postoptimize=False)
dcr2tapn.timed = True
dcr2tapn.initialize_helper_struct(G)
dcr2tapn.mapping_exceptions = timed_exceptional_cases.TimedExceptionalCases(dcr2tapn.event_to_deadline_map)
dcr2tapn.preoptimizer = timed_preoptimizer.TimedPreoptimizer()
G = dcr2tapn.preoptimizer.remove_un_executable_events_from_dcr(G)
dcr2tapn.preoptimizer.preoptimize_based_on_exceptional_cases(G, dcr2tapn.mapping_exceptions)
master_df, marking = dcr2tapn.map_events(G)
master_df = dcr2tapn.mapping_exceptions.map_exceptional_cases_between_events(master_df)
G

In [ ]:
# Function to copy rows of a specified event key n times
def copy_event_rows(src, trg, master_df, relation_table):
    needed_columns = master_df.loc[src][trg].columns
    needed_cases = relation_table[needed_columns]
    n = len(needed_cases)
    m = len(master_df.loc[src])
    repeated_df = pd.DataFrame(np.tile(master_df.loc[src], (n, 1)), columns=master_df.columns, index=pd.MultiIndex.from_product([[src], [i for i in range(n * m)]]))
    repeated_relation_df = pd.DataFrame(np.repeat(needed_cases, m, axis=0), columns=needed_cases.columns,
                                        index=pd.MultiIndex.from_product([[src], [i for i in range(n * m)]]))

    repeated_df.iloc[:, repeated_df.columns.get_level_values(0) == trg] = repeated_relation_df
    master_df = master_df.drop(src, level=0)
    master_df = pd.concat([master_df, repeated_df])
    return master_df


def prepare_timed_case(event_place, case, case_others, copy_to_all_index, copy_pairwise_index, event_to_deadline):
    pend_place = f'Re_{event_place}'
    pend_others = [f'Re_{v}' for k, v in event_to_deadline.items()]
    pend_others.remove(pend_place)
    pend_excl_place = f'Rex_{event_place}'
    pend_excl_others = [f'Rex_{v}' for k, v in event_to_deadline.items()]
    pend_excl_others.remove(pend_excl_place)
    case = case.rename({'Re': pend_place, 'Rex': pend_excl_place}, axis='columns')
    x = len(pend_others)
    cols = pend_others
    cols.extend(pend_excl_others)
    case_all = pd.DataFrame(np.repeat(case_others.loc[copy_to_all_index], x, axis=1), index=copy_to_all_index, columns=cols)
    mat = case_others.loc[copy_pairwise_index].to_numpy()
    idx = deepcopy(copy_pairwise_index)
    idx.extend(copy_pairwise_index)
    ar = np.zeros([s * x for s in mat.shape], mat.dtype)
    for i in range(x):
        ar[i::x, i::x] = mat
    case_pairwise = pd.DataFrame(ar, columns=cols, index=idx)

    case = pd.concat([case, case_all], axis=1).fillna(0)
    case = case.astype(int)
    case = pd.concat([case, case.loc[copy_pairwise_index * (x - 1)]])
    case_pairwise = case_pairwise.sort_index()
    case.loc[copy_pairwise_index, case_pairwise.columns] = case_pairwise
    return case

In [ ]:
event = 'D'
event_prime = 'B'
case = pd.DataFrame([[3,0,0,0],
                     [5,0,6,7],
                     [5,0,0,4]], columns=['In', 'Ex', 'Re', 'Rex'])
case_others = pd.DataFrame([[0,4]],index=[3],columns=['Re','Rex'])
copy_to_all_index = [1]
needed_columns = master_df.loc[event][event_prime].columns

# needed_columns = [c.split('_')[0] for c in needed_columns]
# needed_columns = set([c.split('_')[0] if '_' in c else c for c in needed_columns])

if 'Re' not in needed_columns and 'Rex' not in needed_columns:
    case = case.drop(index=[1])

In [ ]:
copy_event_rows(event, event_prime, master_df, case)

In [ ]:
event = 'A'
event_prime = 'B'

case = pd.DataFrame([[3, 0, 5, 0],
                     [3, 0, 1, 0],
                     [3, 0, 3, 0],
                     [4, 0, 0, 5],
                     [4, 0, 0, 1],
                     [4, 0, 0, 3]], columns=['In', 'Ex', 'Re', 'Rex'])
case_others = pd.DataFrame([[4, 0],
                            [2, 0],
                            [0, 4],
                            [0, 2]],index=[0,1,3,4],columns=['Re','Rex'])
copy_to_all_index = [0,3]
copy_pairwise_index = [1,4]

needed_columns = master_df.loc[event][event_prime].columns
needed_columns = set([c.split('_')[0] if '_' in c else c for c in needed_columns])
if 'Re' not in needed_columns:
    case = case.drop(index=[0,1,2])
    case_others = case_others.drop(index=[0,1])
    copy_to_all_index.remove(0)
    copy_pairwise_index.remove(1)
if 'In' not in needed_columns and 'Rex' not in needed_columns:
    case = case.drop(index=[3,4,5])
    case_others = case_others.drop(index=[3,4])
    copy_to_all_index.remove(3)
    copy_pairwise_index.remove(4)

def prepare_timed_case(event, event_prime, case):
    event_place =  event_to_deadline_map[event_prime][event]
    pend_place = f'Re_{event_place}'
    pend_others = [f'Re_{v}' for k,v in event_to_deadline_map[event_prime].items()]
    pend_others.remove(pend_place)
    pend_excl_place = f'Rex_{event_place}'
    pend_excl_others = [f'Rex_{v}' for k,v in event_to_deadline_map[event_prime].items()]
    pend_excl_others.remove(pend_excl_place)
    case = case.rename({'Re': pend_place,'Rex': pend_excl_place}, axis='columns')
    x = len(pend_others)
    cols = pend_others
    cols.extend(pend_excl_others)
    case_all = pd.DataFrame(np.repeat(case_others.loc[copy_to_all_index], x,axis=1),index=copy_to_all_index,columns=cols)
    mat = case_others.loc[copy_pairwise_index].to_numpy()
    idx = deepcopy(copy_pairwise_index)
    idx.extend(copy_pairwise_index)
    ar = np.zeros([s * x for s in mat.shape], mat.dtype)
    for i in range(x):
        ar[i::x, i::x] = mat
    case_pairwise = pd.DataFrame(ar,columns=cols,index=idx)
    
    case = pd.concat([case,case_all],axis=1).fillna(0)
    case = case.astype(int)
    case = pd.concat([case,case.loc[copy_pairwise_index*(x-1)]])
    case_pairwise = case_pairwise.sort_index()
    case.loc[copy_pairwise_index, case_pairwise.columns] = case_pairwise
    return case
copy_event_rows(event, event_prime, master_df, case)

In [ ]:
base_case = pd.DataFrame([[3, 3, 4, 0],
                       [3, 5, 4, 0],
                       [3, 5, 2, 0],
                       [3, 3, 2, 0]], columns=['In', 'Ex', 'Re', 'Rex'],dtype=int)
n = 3
rows = []
col_vals = [[] for i in range(n)]
for i in range(n):
    rows.extend([2,3])
    base_case[f'Re{i}'] = base_case['Re']
    for k in range(n):
        if k == i:
            col_vals[i].extend([2,2])
        else:
            col_vals[i].extend([0,0])
rrrr= [0,1]
rrrr.extend(rows)
new_bc = base_case.loc[rrrr]
new_bc = new_bc.reset_index(drop=True)
new_bc = new_bc.astype(int)
col_vals_df = pd.DataFrame(col_vals,dtype=int).T
col_vals_df.columns = ['Re0','Re1','Re2']
col_vals_df.index = col_vals_df.index + 2
new_bc.loc[2:,['Re0','Re1','Re2']] = col_vals_df
base_case_rows = set(new_bc.index)

In [ ]:
new_bc.iloc[2::2]

In [ ]:
new_bc

In [ ]:
n = len(base_case_rows)
[i for i in range(3,n,2)]

In [ ]:
new_bc.iloc[3::2]

In [ ]:
if self.preoptimize:
    # this tells which rows to create in the base case: 0 = event, 1 = init, 2 = initpend, 3 = pend
    base_case_rows = {0}
    if default_make_exec and event not in G['marking']['executed'] and event not in self.preoptimizer.no_init_t:
        base_case_rows.add(1)
    if default_make_exec and default_make_pend and event not in self.preoptimizer.no_initpend_t:
        # add all even rows >= 2
        base_case_rows.add(2)
    if default_make_pend:
        # add all odd rows >= 3
        base_case_rows.add(3)

In [ ]:
from pm4py.objects.petri_net.utils import petri_utils as pn_utils
from pm4py.objects.petri_net.obj import InhibitorNet, Marking

In [ ]:
res_pn = InhibitorNet(name='Petri Net from Dcr')
res_m = Marking()
p_dict = {}
for event, place_type in master_df.columns:
    if place_type != 'No':
        p = InhibitorNet.Place(name=f'{event}_{place_type}')
        p_dict[(event, place_type)] = p
        res_pn.places.add(p)
        if place_type in marking[event]:
            res_m[p] = marking[event][place_type]
for event, idx in master_df.index:
    t = InhibitorNet.Transition(name=f'{event}_{idx}',label=f'{event}_{idx}_label')
    res_pn.transitions.add(t)
    for (event_prime,place_type), arc_type in dict(master_df.loc[(event,idx)]).items():
        if arc_type > 0:
            match arc_type:
                case 1: # TtoP -->
                    pn_utils.add_arc_from_to_with_check(t, p_dict[(event_prime, place_type)], res_pn)
                case 2: # PtoT <--
                    pn_utils.add_arc_from_to_with_check(p_dict[(event_prime, place_type)], t, res_pn)
                case 3: # Read Both <-->
                    pn_utils.add_arc_from_to_with_check(p_dict[(event_prime, place_type)], t, res_pn)
                    pn_utils.add_arc_from_to_with_check(t, p_dict[(event_prime, place_type)], res_pn)
                case 4: # Inhib o--
                    pn_utils.add_arc_from_to_with_check(p_dict[(event_prime, place_type)], t, res_pn, type='inhibitor')
                case 5: # TtoPandInhib o-->
                    pn_utils.add_arc_from_to_with_check(t, p_dict[(event_prime, place_type)], res_pn)
                    pn_utils.add_arc_from_to_with_check(p_dict[(event_prime, place_type)], t, res_pn, type='inhibitor')

In [ ]:
include_case = pd.DataFrame([[3, 0, 0, 0],
                              [5, 0, 1, 2],
                              [5, 0, 0, 4]], columns=['In', 'Ex', 'Re', 'Rex'])
src = 'open'
trg = 'leave1'
needed_columns = master_df.loc[src][trg].columns
needed_include_cases = include_case[needed_columns]
n = len(needed_include_cases)
m = len(master_df.loc[src])
repeated_df = pd.DataFrame(np.tile(master_df.loc[src],(n,1)),columns = master_df.columns,index=pd.MultiIndex.from_product([[src],[i for i in range(n*m)]]))
repeated_relation_df = pd.DataFrame(np.repeat(needed_include_cases,m,axis=0),columns = needed_include_cases.columns,index=pd.MultiIndex.from_product([[src],[i for i in range(n*m)]]))
repeated_df.iloc[:,repeated_df.columns.get_level_values(0)==trg] = repeated_relation_df
master_df = master_df.drop(src, level=0)
master_df = pd.concat([master_df,repeated_df])

In [ ]:
backup = deepcopy(master_df)

In [ ]:
# self exception cases: These replace the base case.
# this is handled differently now
# self_include = base_case
self_exclude = pd.DataFrame([[2,3,4,0],
                             [2,5,4,0],
                             [2,5,2,0],
                             [2,3,2,0]], columns=['In','Ex','Re','Rex'])
self_response = pd.DataFrame([[3,3,5,0],
                              [3,5,5,0],
                              [3,5,3,0],
                              [3,3,3,0]], columns=['In','Ex','Re','Rex'])
self_condition = pd.DataFrame([[3,3,4,0],
                               [3,3,2,0]], columns=['In','Ex','Re','Rex'])
self_response_exclude = pd.DataFrame([[2,3,4,1],
                                      [2,5,4,1],
                                      [2,5,2,1],
                                      [2,3,2,1]], columns=['In','Ex','Re','Rex'])

def update_event_base_case_to_self_exception(base_case, event, self_exceptions):
    event_base_case = deepcopy(base_case)
    
    return event_base_case

In [ ]:
# Step 1: Arc pattern tables
# base case
# row index meaning: 0 = event, 1 = init, 2 = initpend, 3 = pend
base_case = pd.DataFrame([[3,3,4,0],
                          [3,5,4,0],
                          [3,5,2,0],
                          [3,3,2,0]], columns=['In','Ex','Re','Rex'])
# core dcr 4 relations
include_case = pd.DataFrame([[3,0,0,0],
                             [5,0,1,2],
                             [5,0,0,4]],columns=['In','Ex','Re','Rex'])
exclude_case = pd.DataFrame([[4,0,0,0],
                             [2,0,4,0],
                             [2,0,2,1]],columns=['In','Ex','Re','Rex'])
response_case = pd.DataFrame([[3,0,5,0],
                             [3,0,3,2],
                             [4,0,0,5],
                             [4,0,0,3]],columns=['In','Ex','Re','Rex'])
condition_case = pd.DataFrame([[3,3,0,0]],columns=['In','Ex','Re','Rex'])
# relation exception cases
condition_include_case = pd.DataFrame([[3,3,0,0],
                                       [5,4,1,2],
                                       [5,4,0,4],
                                       [5,3,1,2],
                                       [5,3,0,4]],columns=['In','Ex','Re','Rex'])
condition_exclude_case = pd.DataFrame([[2,3,4,0],
                                       [2,3,2,1],
                                       [4,4,0,0],
                                       [4,3,0,0]],columns=['In','Ex','Re','Rex'])
condition_response_case = pd.DataFrame([[3,3,5,0],
                                        [3,3,3,0],
                                        [4,4,0,5],
                                        [4,4,0,3],
                                        [4,3,0,5],
                                        [4,3,0,3]],columns=['In','Ex','Re','Rex'])
response_include_case = pd.DataFrame([[3,0,5,0],
                                      [3,0,3,0],
                                      [5,0,5,4],
                                      [5,0,1,2]],columns=['In','Ex','Re','Rex'])
response_exclude_case = pd.DataFrame([[2,0,4,1],
                                      [2,0,2,1],
                                      [4,0,0,5],
                                      [4,0,0,3]],columns=['In','Ex','Re','Rex'])
condition_response_include_case = pd.DataFrame([[3,3,5,0],
                                                [3,3,3,0],
                                                [5,4,1,2],
                                                [5,4,5,4],
                                                [5,3,1,2],
                                                [5,3,5,4]],columns=['In','Ex','Re','Rex'])
condition_response_exclude_case = pd.DataFrame([[2,3,4,1],
                                                [2,3,2,1],
                                                [4,4,0,5],
                                                [4,4,0,3],
                                                [4,3,0,5],
                                                [4,3,0,3]],columns=['In','Ex','Re','Rex'])
# extended dcr with no response and milestones
noresponse_case = pd.DataFrame([[3,0,0,0],
                             [5,0,1,2],
                             [5,0,0,4]],columns=['In','Ex','Re','Rex'])
milestone_case = pd.DataFrame([[3,0,0,0],
                             [5,0,1,2],
                             [5,0,0,4]],columns=['In','Ex','Re','Rex'])
# Step 2: Create a dictionary of small dataframes
events = ['A','B','C','D']
base_case_dict = {}
for event in events:
    base_case_dict[event] = deepcopy(base_case)

# Step 3: Create a multilevel column index for each dataframe
for event in base_case_dict:
    base_case_dict[event].columns = pd.MultiIndex.from_product([[event], base_case_dict[event].columns])

# Step 4: Determine the padding and shift indices
max_length = len(base_case)
padded_dfs = []

for i, event in enumerate(events):
    shifted_df = base_case_dict[event]
    shifted_df.index = shifted_df.index + i * max_length
    
    # Pad the dataframe with zeros
    padded_df = shifted_df.reindex(range(len(events) * max_length), fill_value=0)
    padded_dfs.append(padded_df)

# Step 5: Concatenate the dataframes and create a multi-index for rows
master_df = pd.concat(padded_dfs, axis=1)

# Create the multi-index for rows
row_tuples = []
for i, event in enumerate(events):
    row_tuples.extend([(event, idx) for idx in range(max_length)])

multi_index = pd.MultiIndex.from_tuples(row_tuples, names=["Event", "Index"])
master_df.index = multi_index
# Function to copy rows of a specified event key n times
def copy_event_rows(master_df, source_event_key, target_event_key, relation_table):
    # Filter rows by event key using .loc
    rows_for_event = master_df.loc[source_event_key]
    n = len(relation_table.index)
    # Repeat these rows n times
    repeated_rows = pd.concat([rows_for_event] * n, ignore_index=True)
    # Add the event key to the new index
    new_index = pd.MultiIndex.from_product([[source_event_key], range(n * len(rows_for_event))], names=["Event", "Index"])
    repeated_rows.index = new_index
    
    j = int(len(repeated_rows.index)/n)
    for i in range(n):
        col_name = relation_table.columns
    # for col_name in relation_table.columns.to_list():
        row_index = list(range(i*j,(i+1)*j))
        repeated_rows.loc[(source_event_key,row_index),(target_event_key,col_name)] = list(relation_table.loc[i,col_name])
        i += 1
    
    return repeated_rows

source_event_key = 'A'
target_event_key = 'B'
repeated_rows_df = copy_event_rows(master_df, source_event_key, target_event_key, include_case)

# Remove the existing rows for the specified event key from the master_df
master_df = master_df.drop(source_event_key)

# Concatenate the repeated rows with the modified master_df
master_df = pd.concat([master_df, repeated_rows_df]).sort_index(level=["Event", "Index"])

source_event_key = 'A'
target_event_key = 'C'
repeated_rows_df = copy_event_rows(master_df, source_event_key, target_event_key, exclude_case)

# Remove the existing rows for the specified event key from the master_df
master_df = master_df.drop(source_event_key)

# Concatenate the repeated rows with the modified master_df
master_df = pd.concat([master_df, repeated_rows_df]).sort_index(level=["Event", "Index"])

print(master_df)

In [ ]:
repeated_rows_df

In [ ]:
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.dcr.utils.utils import nested_groups_and_sps_to_flat_dcr
from pm4py.objects.conversion.dcr import converter as dcr_to_tapn
from pm4py.vis import view_petri_net
from pm4py.objects.dcr.obj import dcr_template
from pm4py.objects.petri_net.utils import reachability_graph
from pm4py.visualization.transition_system import visualizer as ts_visualizer
from pm4py.objects.petri_net.timed_arc_net import semantics as tapn_semantics
from pm4py.objects.petri_net.inhibitor_reset import semantics as in_semantics
from pm4py.objects.petri_net.utils.petri_utils import remove_arc, remove_transition, remove_place, add_arc_from_to_with_check, pre_set, post_set, get_arc_type
from pm4py.objects.petri_net.utils import reduction, murata
from pm4py.objects.petri_net.exporter import exporter as pnml_exporter
from pm4py.objects.petri_net import properties
from copy import deepcopy
from pm4py.objects.petri_net.timed_arc_net import semantics as tapn_semantics, obj as tapn_obj
from pm4py.objects.petri_net.inhibitor_reset import semantics as inhib_semantics
from pm4py.objects.petri_net.utils import petri_utils as pn_utils

In [ ]:
path = 'models/pula.xml'
dcr_dict = dcr_importer.apply('models/pula.xml',parameters={'as_dcr_object':False,'labels_as_ids':True})

In [ ]:
print(os.path.exists(path))  # This should return True if the file exists

In [ ]:
example = 'models/RunningExampleTAPNoriginal.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':False,'labels_as_ids':True})
dcr_dict
net, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True,
                       'postoptimize':True,                                   'map_unexecutable_events':True,
                        'debug':False,'tapn_path':'models/RunningExampleTAPNoriginal.tapn'})
view_petri_net(net,m)

In [ ]:
example = 'models/rail_example.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
nested_groups_and_sps_to_flat_dcr(dcr_dict)
dcr_dict = dcr_dict.obj_to_template()
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True, 'postoptimize':True, 'map_unexecutable_events':False, 'debug':False,'tapn_path':'models/rail_example.tapn'})

In [ ]:
example = 'models/rail_example_simpler.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':True,'labels_as_ids':True})
nested_groups_and_sps_to_flat_dcr(dcr_dict)
dcr_dict = dcr_dict.obj_to_template()
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True, 'postoptimize':True, 'map_unexecutable_events':False, 'debug':False,'tapn_path':'models/rail_example_simpler.tapn'})

# Irrelevant for now

In [ ]:
net2, m2, _ = murata.apply_reduction(net,m,set())
view_petri_net(net2,m2)

In [ ]:
print(f'P: {len(net.places)} T: {len(net.transitions)} A: {len(net.arcs)}')
print(f'P: {len(net2.places)} T: {len(net2.transitions)} A: {len(net2.arcs)}')

In [ ]:
pnml_exporter.apply(net2,m2,'models/murata_net_example.tapn',variant=pnml_exporter.TAPN)

In [ ]:
example = 'models/test4.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':False,'labels_as_ids':True})
# dcr_dict = nested_groups_and_sps_to_flat_dcr(dcr_dict)
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_INHIBITOR_NET, parameters={'preoptimize':True,
               'postoptimize':True,                                                                                  'map_unexecutable_events':False,
                'debug':True, 'tapn_path':'models/test4_untimed.tapn'})

In [ ]:
trans_sys = reachability_graph.construct_reachability_graph(tapn, m, use_trans_name=True,
parameters={'petri_semantics': tapn_semantics.TimedArcSemantics()})

In [ ]:
ts_visualizer.apply(trans_sys)

In [ ]:
example = 'models/test4.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':False,'labels_as_ids':True})
# nested_groups_and_sps_to_flat_dcr(rail_dcr_labeled)
# dcr_dict = rail_dcr_labeled.obj_to_template()
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True, 'postoptimize':True, 'map_unexecutable_events':False, 'debug':True,'tapn_path':'models/test4.tapn'})

In [ ]:
import os
print(os.getcwd())
os.chdir('/home/vco/Projects/pm4py-dcr/')
print(os.getcwd())
from pm4py.objects.dcr.importer import importer as dcr_importer
from pm4py.objects.conversion.dcr import converter as dcr_to_tapn
example = 'models/rail_example.xml'
dcr_dict = dcr_importer.apply(example,parameters={'as_dcr_object':False,'labels_as_ids':True})
# nested_groups_and_sps_to_flat_dcr(rail_dcr_labeled)
# dcr_dict = rail_dcr_labeled.obj_to_template()
tapn, m = dcr_to_tapn.apply(dcr_dict,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True, 'postoptimize':True, 'map_unexecutable_events':False, 'debug':True,'tapn_path':'models/rail_example.tapn'})

In [ ]:
tapn = tapn_obj.TimedArcNet()
P0 = tapn.Place('P0',properties={properties.AGE_INVARIANT:10})
P1 = tapn.Place('P1')
T0 = tapn.Transition('T0')
tapn.places.add(P0)
tapn.places.add(P1)
tapn.transitions.add(T0)
a0 = pn_utils.add_arc_from_to_with_check(P0, T0, tapn, type=properties.TRANSPORT_ARC)
a1 = pn_utils.add_arc_from_to_with_check(T0, P1, tapn, type=properties.TRANSPORT_ARC)
a0.properties[properties.AGE_MIN] = 5
a0.properties[properties.TRANSPORT_INDEX] = 0
a1.properties[properties.TRANSPORT_INDEX] = 0
m = tapn_obj.TimedMarking()
m[P0] = 1
m.timed_dict[P0] = 0

In [ ]:
view_petri_net(tapn,m)

In [ ]:
m.timed_dict[P0] = 0

In [ ]:
m

In [ ]:
tapn_semantics.enabled_transitions(tapn,m)

In [ ]:
inhib_semantics.enabled_transitions(tapn,m)

In [ ]:
test_dcr = deepcopy(dcr_template)
test_dcr['events'] = {'A','B','C'}
test_dcr['includesTo'] = {'A':{'B'}}
test_dcr['conditionsFor'] = {'C': {'A'}}
test_dcr['responseTo'] = {'A':{'B'}}
test_dcr['excludeTo'] = {'A':{'A'},'B':{'B'}}
test_dcr['marking']['included'] = {'A','C'}

tapn, m = dcr_to_tapn.apply(test_dcr,variant=dcr_to_tapn.Variants.TO_TIMED_ARC_PETRI_NET, parameters={'preoptimize':True,
               'postoptimize':True,                                                                                  'map_unexecutable_events':False,
                'debug':False,                                                                       'tapn_path':'models/test.tapn'})